In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib import parse
from IPython.display import HTML
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from IPython.display import Image
import pandas as pd
import re
import numpy as np
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time

In [ ]:
tc_export_name = "Timecard Export.csv"

In [ ]:
themis_website = "https://thmprod.themis.unimelb.edu.au/OA_HTML/RF.jsp?function_id=39493"

In [ ]:
# Open Selenium Browser
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

In [ ]:
# Set up target rows
export_df = pd.read_csv(tc_export_name, dtype=str)
timecard_rows = export_df[~export_df.isna().apply(lambda x: all(x), axis=1)]
date_pattern = r"^(.+) (Start|End)$"

def build_rows_list(rows_df):
    row_bundle = {}
    row_order = []
    row_order_seen = set()
    for idx in rows_df.index:
        for c in rows_df.columns:
            if idx not in row_bundle.keys():
                row_bundle[idx] = {}
            if "start" in c.lower() or "end" in c.lower():
                date_val, d_row = re.match(date_pattern, c).group(1, 2)
                if date_val not in row_bundle[idx].keys():
                    row_bundle[idx][date_val] = {}
                    if date_val not in row_order_seen:
                        row_order.append(date_val)
                        row_order_seen.add(date_val)
                row_bundle[idx][date_val][d_row] = rows_df.loc[idx, c]
            else:
                if c not in row_bundle[idx].keys():
                    row_bundle[idx][c] = rows_df.loc[idx, c]
                    if c not in row_order_seen:
                        row_order.append(c)
                        row_order_seen.add(c)
                        
    return row_bundle, row_order

def get_row_string(row, row_order):
    # row_order = sorted(list(row.keys()))
    row_strings = []
    for key in row_order:
        if type(row[key]) == dict:
            if (type(row[key]["Start"]) == float or isinstance(row[key]["Start"], np.floating)) and np.isnan(row[key]["Start"]):
                s_val = ''
            else:
                s_val = row[key]["Start"]
            if (type(row[key]["End"]) == float or isinstance(row[key]["End"], np.floating)) and np.isnan(row[key]["End"]):
                e_val = ''
            else:
                e_val = row[key]["End"]
                
            row_strings.append("{},{}".format(s_val, e_val))
        else:
            if (type(row[key]) == float or isinstance(row[key], np.floating)) and np.isnan(row[key]):
                row_strings.append('')
            else:
                row_strings.append("{}".format(row[key]))
    
    return ",".join(row_strings)

full_rows, row_order = build_rows_list(timecard_rows)
full_rows_identifiers = [get_row_string(row, row_order) for row_idx, row in full_rows.items()]

## Set up reading functions.
def read_row_col_select(row, col):
    select_element_dropdown = driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//select'.format(row, col))
    select_dropdown = Select(select_element_dropdown)
    if len(select_dropdown.all_selected_options) == 0:
        return ''
    else: 
        return select_dropdown.all_selected_options[0].text

def read_row_col_val_delay(row, col):
    text_element = driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col))
    return text_element.get_attribute("value")

def read_row_col_val(row, col):
    text_element = driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col))
    return text_element.get_attribute("value")

def read_row_col_day(row, col):
    text_element_start = driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[1]//td//input'.format(row, col))
    text_element_end = driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[2]//td//input'.format(row, col))
    return {"Start": text_element_start.get_attribute("value"), "End": text_element_end.get_attribute("value")}

row_read_processing_rules = {
    "Casual Contract Reference": (lambda row, col: read_row_col_select(row, col)),
    "Hours Type": (lambda row, col: read_row_col_select(row, col)),
    "Payment Type": (lambda row, col: read_row_col_val_delay(row, col)),
    "Job Performed": (lambda row, col: read_row_col_select(row, col)),
    "Approval ID": (lambda row, col: read_row_col_select(row, col)),
    "Coy": (lambda row, col: read_row_col_val_delay(row, col)),
    "Budg": (lambda row, col: read_row_col_val_delay(row, col)),
    "Cost": (lambda row, col: read_row_col_val_delay(row, col)),
    "Proj": (lambda row, col: read_row_col_val_delay(row, col)),
    "Pur": (lambda row, col: read_row_col_val_delay(row, col)),
    "Act": (lambda row, col: read_row_col_val_delay(row, col)),
    "Loc": (lambda row, col: read_row_col_val_delay(row, col)),
}

day_rules = [(row_name, (lambda row, col: read_row_col_day(row, col))) for row_name in row_order if row_name not in row_read_processing_rules.keys()]

for row_name, rule in day_rules:
    row_read_processing_rules[row_name] = rule

row_read_processing_rules["Comments"] = lambda row, col: read_row_col_val(row, col)

wait = WebDriverWait(driver, 60 * 15)

# Write rules.
def write_row_col_select(row, col, selection):
    select_element_dropdown = driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//select'.format(row, col))
    select_dropdown = Select(select_element_dropdown)
    
    option = None
    for candidate_option in select_dropdown.options:
        if candidate_option.accessible_name == selection:
            option = candidate_option
    assert option is not None
    select_dropdown.select_by_visible_text(selection)

def write_row_col_val_delay(row, col, text):
    if read_row_col_val_delay(row, col) == text:
        # Already right value, skip over.
        return
    time.sleep(10)
    wait.until(lambda d: d.execute_script('return document.readyState') == 'complete')
    
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).click()
    # Avoid double up if re-run
    if not read_row_col_val_delay(row, col) == '':
        driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).clear()
        time.sleep(10)
        wait.until(lambda d: d.execute_script('return document.readyState') == 'complete')
        driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).click()
    time.sleep(10)
    wait.until(lambda d: d.execute_script('return document.readyState') == 'complete')
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).send_keys(text)
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).click()
    time.sleep(1)
    wait.until(lambda d: d.execute_script('return document.readyState') == 'complete')

def write_row_col_val(row, col, text):
    # Make sure we're ok to type
    time.sleep(1)
    wait.until(lambda d: d.execute_script('return document.readyState') == 'complete')
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).click()
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).clear()
    # Make sure we're ok to type
    time.sleep(1)
    wait.until(lambda d: d.execute_script('return document.readyState') == 'complete')
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).click()
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).send_keys(text)

def write_row_col_day(row, col, entry):
    start_data = entry["Start"]
    if start_data is None or not type(start_data) == str:
        start_data = ''
    end_data = entry["End"]
    if end_data is None or not type(end_data) == str:
        end_data = ''
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[1]//td//input'.format(row, col)).click()
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[1]//td//input'.format(row, col)).clear()
    time.sleep(1)
    wait.until(lambda d: d.execute_script('return document.readyState') == 'complete')
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[1]//td//input'.format(row, col)).click()
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[1]//td//input'.format(row, col)).send_keys(start_data)
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[2]//td//input'.format(row, col)).click()
    time.sleep(1)
    wait.until(lambda d: d.execute_script('return document.readyState') == 'complete')
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[2]//td//input'.format(row, col)).clear()
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[2]//td//input'.format(row, col)).click()
    driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[2]//td//input'.format(row, col)).send_keys(end_data)
    # return {"Start": text_element_start.text, "End": text_element_end.text}

row_write_processing_rules = {
    "Casual Contract Reference": (lambda row, col, selection: write_row_col_select(row, col, selection)),
    "Hours Type": (lambda row, col, selection: write_row_col_select(row, col, selection)),
    "Payment Type": (lambda row, col, text: write_row_col_val_delay(row, col, text)),
    "Job Performed": (lambda row, col, selection: write_row_col_select(row, col, selection)),
    "Approval ID": (lambda row, col, selection: write_row_col_select(row, col, selection)),
    "Coy": (lambda row, col, text: write_row_col_val_delay(row, col, text)),
    "Budg": (lambda row, col, text: write_row_col_val_delay(row, col, text)),
    "Cost": (lambda row, col, text: write_row_col_val_delay(row, col, text)),
    "Proj": (lambda row, col, text: write_row_col_val_delay(row, col, text)),
    "Pur": (lambda row, col, text: write_row_col_val_delay(row, col, text)),
    "Act": (lambda row, col, text: write_row_col_val_delay(row, col, text)),
    "Loc": (lambda row, col, text: write_row_col_val_delay(row, col, text)),
}

day_w_rules = [(row_name, (lambda row, col, entry: write_row_col_day(row, col, entry))) for row_name in row_order if row_name not in row_write_processing_rules.keys()]

for row_name, rule in day_w_rules:
    row_write_processing_rules[row_name] = rule

row_write_processing_rules["Comments"] = lambda row, col, text: write_row_col_val(row, col, text)

In [ ]:
# Load page
driver.get(themis_website)

### Selenium Viewer

In [ ]:
# This small bit lets you see what is shown on the screen, useful for checking for errors
#   and whether to move onto the next step.
#driver.execute_script("window.scrollTo(500, 1600)")
driver.execute_script("window.scrollTo(0, 0)")
display(Image(data=driver.get_screenshot_as_png()))

### Okta Login
If you haven't logged in yet, you'll need to do the Okta challenge.

In [ ]:
# Add username in.
driver.find_element(By.NAME, "identifier").click()
MY_USERNAME = "..."
driver.find_element(By.NAME, "identifier").send_keys(MY_USERNAME)

In [ ]:
# Add password in.
driver.find_element(By.NAME, "credentials.passcode").click()
MY_PASSWORD = "..."
driver.find_element(By.NAME, "credentials.passcode").send_keys(MY_PASSWORD)

In [ ]:
# Log in button click!
driver.find_element(By.CLASS_NAME, "button-primary").click()

In [ ]:
# Get push notification from Okta
driver.find_element(By.XPATH, '//h3[text()="Get a push notification"]/../../div[contains(@class, "authenticator-button")]/a').click()

In [ ]:
# Took too long.
# driver.find_element(By.CLASS_NAME, "js-go-back").click()

In [ ]:
# If you aren't sure if your password is going in correctly.
# driver.find_element(By.CLASS_NAME, "button-show").click()

### Themis Login
This set of cells log you into Themis. Themis has a number of bugs, hopefully the script should avoid them and enter entries fast enough to avoid timeouts. But you may find it useful to return here.

In [ ]:
# Add username in.
driver.find_element(By.ID, "unamebean").click()
driver.find_element(By.ID, "unamebean").send_keys(MY_USERNAME)

In [ ]:
# Add password in.
driver.find_element(By.ID, "pwdbean").click()
driver.find_element(By.ID, "pwdbean").send_keys(MY_PASSWORD)

In [ ]:
# This can be used to delete text from a field.
# driver.find_element(By.ID, "pwdbean").click()
# driver.find_element(By.ID, "pwdbean").clear()

In [ ]:
driver.find_element(By.ID, "SubmitButton").click()

### Themis Timecard Move
The following steps open some folders and then move to the "Create Timecards" section. There are lots of ways which could lead to these folders being open, and clicking on them toggles them, so if something goes wrong in these steps, try running the viewer cell to work out which cells need to be run

In [ ]:
driver.find_element(By.XPATH, "//a[text()='UOM Staff Self Service']").click()

In [ ]:
driver.find_element(By.XPATH, "//a[text()='Timecard']").click()

In [ ]:
driver.find_element(By.XPATH, "//a[text()='Create Timecard']").click()

### Themis Timecard Select
This step selects the timecard to fill, to avoid mis-entry, this is set explicitly instead of being derived from the timecard export.

The ~ at the end of a timecard suggests some rows have been filled. So it is best to keep this out of the target timecard name (and it's best to remove it as soon as you've finished entering the timecard).

In [ ]:
TIMECARD_TO_FILL = "July 08, 2023 - July 21, 2023"
# TIMECARD_TO_FILL = "June 08, 2023 - July 21, 2023~"

In [ ]:
select_element = driver.find_element(By.NAME, "TimecardPeriodList")
select = Select(select_element)
option = None
for candidate_option in select.options:
    if candidate_option.accessible_name == TIMECARD_TO_FILL:
        # print(candidate_option.accessible_name)
        option = candidate_option
assert option is not None
select.select_by_visible_text(TIMECARD_TO_FILL)

### Row Fill
This section fills in the rows based on the target. Rows matching existing target rows will be excluded.

In [ ]:
def fill_row(row_idx, row_content):
    for col in row_order:
        # 1-indexed + skip over heading row.
        html_row = row_idx + 2
        # 1-indexed
        html_col = 1 + col_map[col]
        row_write_processing_rules[col](html_row, html_col, row_content[col])
    driver.find_element(By.XPATH, "//button[@title='Save']").click()
    time.sleep(1)
    wait.until(lambda d: d.execute_script('return document.readyState') == 'complete')
    driver.find_element(By.XPATH, "//button[text()='Add Another Row']").click()
    time.sleep(1)
    wait.until(lambda d: d.execute_script('return document.readyState') == 'complete')

In [ ]:
def get_current_state_bundles():
    current_state_bundles = []

    for row_num in range(all_row_count):
        row_entry = {}
        for col in row_order:
            col_to_read = col_map[col]
            row_entry[col] = row_read_processing_rules[col](2 + row_num, 1 + col_to_read)
        current_state_bundles.append(row_entry)
    return current_state_bundles

def get_status(current_state_bundles):
    current_row_states = [get_row_string(cs, row_order) for cs in current_state_bundles]

    unfilled_rows = set(full_rows_identifiers) - set(current_row_states)
    filled_rows = set(full_rows_identifiers) - unfilled_rows
    
    return current_row_states, unfilled_rows, filled_rows

def get_current_col_data():
    page_soup = BeautifulSoup(driver.page_source, "html.parser")
    relevant_table = page_soup.find('span', text=' Casual Contract Reference ').find_parent('table').find_parent('table')
    relevant_tbody = relevant_table.tbody

    cols = {}

    selects = set(["Casual Contract Reference", "Hours Type", "Job Performed", "Approval ID"])

    for row in relevant_tbody.children:
        for cell_idx, cell in enumerate(row.children):
            if cell_idx not in cols.keys():
                cols[cell_idx] = {}
                cols[cell_idx]["name"] = cell.text.strip()
                cols[cell_idx]["entries"] = []
            else:
                entry = []
                if cols[cell_idx]["name"] in selects:
                    selected = None
                    #display(HTML(str(cell)))
                    for cell_option in cell.find_all('option', selected=True):
                        # print(cell_option)
                        entry = cell_option.text
                        selected = entry
                    if selected is None:
                        entry = ''
                elif len(cell.table.find_all('table')) > 1:
                    for table in cell.table.table.find_all('table'):
                        entry.append(table.text)
                else:
                    entry = cell.text
                cols[cell_idx]["entries"].append(entry)
    return cols


start_time = time.time()
row_idxes = sorted(list(full_rows.keys()))

stale = True

for row_idx in row_idxes:
    if stale:
        cols = get_current_col_data()
        col_map = {col_data["name"]: col_idx for col_idx, col_data in cols.items() if col_data["name"] in row_read_processing_rules.keys()}
        
        all_row_count = min([len(col["entries"]) for col in cols.values()])
        
        current_state_bundles = get_current_state_bundles()

        current_row_states, unfilled_rows, filled_rows = get_status(current_state_bundles)
        stale = False
    
    if full_rows_identifiers[row_idx] in filled_rows:
        print("Already filled: Row #", row_idx)
        continue
    else:
        to_fill_row = None
        for html_idx, row_option in enumerate(current_row_states):
            if row_option not in filled_rows:
                to_fill_row = html_idx
                break
        
        assert to_fill_row is not None
        print("Filling: Row #", row_idx, "(Checkpoint time: {})".format(time.time() - start_time))
        fill_row(to_fill_row, full_rows[row_idx])
        stale = True
print("Total Duration:", time.time() - start_time)

In [ ]:
# %pdb

### Miscellaneous Extra Code
This isn't really standard required code, but may be useful to have on hand if something went wrong in your input and you want to do manual changes.

In [ ]:
# Clear Delayed Column
#row, col = 2, 8
#driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).click()
#driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//input'.format(row, col)).clear()
# Clear Day Entry
#row, col = 2, 16
#driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//td[1]//input'.format(row, col)).click()
#driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//td[1]//input'.format(row, col)).clear()
#driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[1]/td//input'.format(row, col)).click()
#driver.find_element(By.XPATH, '//span[text()=" Casual Contract Reference "]/../../../../../../../tr[{}]/td[{}]//table//table//tr[2]/td//input'.format(row, col)).clear()

In [ ]:
# Check other parts of the view
#driver.execute_script("window.scrollTo(1600, 1500)")
#driver.execute_script("window.scrollTo(1800, 1100)")
#display(Image(data=driver.get_screenshot_as_png()))

In [ ]:
# Close extra window - useful if you enter a code missing some parts (e.g. 1 instead of 01)
#print(driver.window_handles)
#driver.switch_to.window(driver.window_handles[1])
#driver.close()
#driver.switch_to.window(driver.window_handles[0])

In [ ]:
# If Themis session expires, you may need to click a "here" text button.
# driver.find_element(By.XPATH, "//font[text()=' here ']").click()